<a href="https://www.kaggle.com/code/prasannakasar/image-captioning?scriptVersionId=220002024" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
# df = pd.read_csv("/kaggle/input/flickr8k/captions.txt", sep=",")

In [3]:
BASE_PATH = '../input/coco-2017-dataset/coco2017'

In [4]:
import json

with open(f'{BASE_PATH}/annotations/captions_train2017.json', 'r') as f:
    data = json.load(f)
    data = data['annotations']

img_cap_pairs = []

for sample in data:
    img_name = '%012d.jpg' % sample['image_id']
    img_cap_pairs.append([img_name, sample['caption']])

df = pd.DataFrame(img_cap_pairs, columns=['image', 'caption'])
df['image'] = df['image'].apply(
    lambda x: f'{BASE_PATH}/train2017/{x}'
)
df = df.reset_index(drop=True)
df.head()

,image,caption
0,../input/coco-2017-dataset/coco2017/train2017/000000203564.jpg,A bicycle replica with a clock as the front wheel.
1,../input/coco-2017-dataset/coco2017/train2017/000000322141.jpg,A room with blue walls and a white sink and door.
2,../input/coco-2017-dataset/coco2017/train2017/000000016977.jpg,A car that seems to be parked illegally behind a legally parked car
3,../input/coco-2017-dataset/coco2017/train2017/000000106140.jpg,A large passenger airplane flying through the air.
4,../input/coco-2017-dataset/coco2017/train2017/000000106140.jpg,There is a GOL plane taking off in a partly cloudy sky.


In [5]:
len(df)

591753

In [6]:
df['cleaned_caption'] = df['caption'].apply(lambda sentence : [word.lower() for word in sentence.split(" ") if word.isalpha()])
df['cleaned_caption'] = df['cleaned_caption'].apply(lambda lis : [word for word in lis if len(word) > 1])
df['cleaned_caption'] = df['cleaned_caption'].apply(lambda lis : ['<start>'] + lis + ['<end>'])

In [7]:
df.head(3)

,image,caption,cleaned_caption
0,../input/coco-2017-dataset/coco2017/train2017/000000203564.jpg,A bicycle replica with a clock as the front wheel.,"[<start>, bicycle, replica, with, clock, as, the, front, <end>]"
1,../input/coco-2017-dataset/coco2017/train2017/000000322141.jpg,A room with blue walls and a white sink and door.,"[<start>, room, with, blue, walls, and, white, sink, and, <end>]"
2,../input/coco-2017-dataset/coco2017/train2017/000000016977.jpg,A car that seems to be parked illegally behind a legally parked car,"[<start>, car, that, seems, to, be, parked, illegally, behind, legally, parked, car, <end>]"


In [8]:
df['seq_len'] = df['cleaned_caption'].apply(lambda x : len(x))
max_len = df['seq_len'].max()
max_len

45

In [9]:
df['cleaned_caption'].apply(len).idxmax()

76733

In [10]:
df['cleaned_caption'] = df['cleaned_caption'].apply(lambda lis : lis + ['<pad>'] * (max_len - len(lis)))

In [11]:
df.head(3)

,image,caption,cleaned_caption,seq_len
0,../input/coco-2017-dataset/coco2017/train2017/000000203564.jpg,A bicycle replica with a clock as the front wheel.,"[<start>, bicycle, replica, with, clock, as, the, front, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",9
1,../input/coco-2017-dataset/coco2017/train2017/000000322141.jpg,A room with blue walls and a white sink and door.,"[<start>, room, with, blue, walls, and, white, sink, and, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",10
2,../input/coco-2017-dataset/coco2017/train2017/000000016977.jpg,A car that seems to be parked illegally behind a legally parked car,"[<start>, car, that, seems, to, be, parked, illegally, behind, legally, parked, car, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",13


In [12]:
word_list = []
df['cleaned_caption'].apply(lambda lis: [word_list.append(word) for word in lis])

0         [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
1         [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
2         [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
3         [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, No

In [13]:
from collections import Counter
word_dict = Counter(word_list)

In [14]:
word_dict = sorted(word_dict, key=word_dict.get, reverse=True)

In [15]:
word_to_index = {word : index for index, word in enumerate(word_dict)}
index_to_word = {index : word for index, word in enumerate(word_dict)}

In [16]:
df['word_token'] = df['cleaned_caption'].apply(lambda lis : [word_to_index[word] for word in lis])

In [17]:
df.head(2)

,image,caption,cleaned_caption,seq_len,word_token
0,../input/coco-2017-dataset/coco2017/train2017/000000203564.jpg,A bicycle replica with a clock as the front wheel.,"[<start>, bicycle, replica, with, clock, as, the, front, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",9,"[1, 348, 3663, 7, 74, 101, 5, 37, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,../input/coco-2017-dataset/coco2017/train2017/000000322141.jpg,A room with blue walls and a white sink and door.,"[<start>, room, with, blue, walls, and, white, sink, and, <end>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>, <pad>]",10,"[1, 52, 7, 53, 614, 8, 20, 152, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [18]:
df.drop(columns=['caption', 'cleaned_caption', 'seq_len'], inplace=True)
df.head(3)

,image,word_token
0,../input/coco-2017-dataset/coco2017/train2017/000000203564.jpg,"[1, 348, 3663, 7, 74, 101, 5, 37, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,../input/coco-2017-dataset/coco2017/train2017/000000322141.jpg,"[1, 52, 7, 53, 614, 8, 20, 152, 8, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,../input/coco-2017-dataset/coco2017/train2017/000000016977.jpg,"[1, 141, 22, 1313, 11, 294, 56, 8027, 123, 10428, 56, 141, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [19]:
train_size = int(0.9*len(df))
test_size = len(df) - train_size
#/kaggle/input/flickr8k/Images

In [20]:
import torch
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import matplotlib.pyplot as plt

class ImageDataset(Dataset):
    def __init__(self, img_dir, dataframe):
        self.img_dir = img_dir
        self.dataframe = dataframe
        self.scaler = transforms.Resize([299, 299])
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]
        
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path)
        if image.mode == "L":
            image = image.convert("RGB")
        t_img = self.normalize(self.to_tensor(self.scaler(image)))
        return t_img, torch.tensor(label)

In [21]:
img_dir = '/kaggle/input/'
dataset = ImageDataset(img_dir, df)

In [22]:
batch_size = 256
vocab_size = len(word_dict)
embed_size = 256
hidden_size = 256
num_layers = 4
learning_rate = 0.001

In [23]:
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        self.inception = models.inception_v3(pretrained=True, aux_logits=True)
        self.inception.aux_logits = False  # Disable it after creation
        self.inception.AuxLogits = None    # Remove the auxiliary classifier
        
        for param in self.inception.parameters():
            param.requires_grad = False
            
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        
        for param in self.inception.fc.parameters():
            param.requires_grad = True
            
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, image): 
        # print(image.size())
        features = self.inception(image)

        features = self.dropout(self.relu(features))

        return features

In [25]:
class LSTMCaptionGenerator(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(LSTMCaptionGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, features, captions):
        embeddings = self.embed(captions)
        embeddings = self.dropout(embeddings)
        # print(f"dim of features after unsqueezzing = ", features.unsqueeze(1).size())
        # print(f"dim of embeddings = ", embeddings.size())
        # repeated_features = features.unsqueeze(1).repeat(1, embeddings.size(1), 1)
        embeddings = torch.cat((features.unsqueeze(1), embeddings[:, :-1, :]), dim=1)
        # embeddings = torch.cat((repeated_features, embeddings), dim=1) 
        # print(f"embedding dim={embeddings.size()}")
        hiddens, _ = self.lstm(embeddings)
        outputs = self.linear(hiddens)
        return outputs

In [26]:
# class CNN_LSTM_model(nn.Module):  # Must inherit from nn.Module
#     def __init__(self, vocab_size, embed_size, hidden_size, num_layers, max_seq_length):
#         super(CNN_LSTM_model, self).__init__()
#         self.CNN_model = CNNFeatureExtractor()  # Initialize CNN feature extractor
#         self.LSTM_model = LSTMCaptionGenerator(vocab_size, embed_size, hidden_size, num_layers, max_seq_length)  # Initialize LSTM model

#     def forward(self, image, captions):
#         features = self.CNN_model(image)  # Get image features from CNN
#         outputs = self.LSTM_model(features, captions)  # Use image features and captions in LSTM
#         return outputs

In [27]:
import torch

device = ""
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

encoder = CNNFeatureExtractor().to(device)
decoder = LSTMCaptionGenerator(embed_size=embed_size, 
                       hidden_size=hidden_size,
                       vocab_size=vocab_size,
                       num_layers=num_layers).to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 162MB/s] 


In [28]:
index_to_word[0]

'<pad>'

In [29]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu_score(predictions, references, idx2word, tokenizer):
    """
    Calculate BLEU score for a batch of predictions vs references.
    predictions: List of predicted captions (list of words).
    references: List of ground truth captions (list of words).
    """
    # Convert predicted captions and ground truth into the format required by BLEU
    # References should be a list of lists, and predictions should be a list of sentences.
    bleu_score = corpus_bleu(references, predictions)
    return bleu_score

In [30]:
from tqdm import tqdm
import torch
import torch.nn as nn
import nltk

def train_and_test(num_epochs):
    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(num_epochs):
        # Training phase
        encoder.train()
        decoder.train()
        total_loss_train = 0
        for image, caption in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training"):
            image = image.to(device)
            caption = caption.to(device)

            # print(image.size())

            optimizer.zero_grad()
            features = encoder(image)
            predicted_caption = decoder(features, caption)
            predicted_caption = predicted_caption.reshape(-1, predicted_caption.size(-1))
            caption = caption.reshape(-1)

            loss = criterion(predicted_caption, caption)
            loss.backward()
            optimizer.step()
            total_loss_train += loss.item()

        print(f"Train Loss at epoch {epoch+1} = {total_loss_train / len(train_loader)}")

        # Testing phase
        encoder.eval()
        decoder.eval()
        total_loss_test = 0
        all_predicted_captions = []  # Store predicted captions
        all_references = []         # Store ground truth captions

        for image, caption in tqdm(test_loader, desc=f"Epoch {epoch+1}/{num_epochs} Testing"):
            image = image.to(device)
            caption = caption.to(device)

            features = encoder(image)
            predicted_caption = decoder(features, caption)
            predicted_caption = predicted_caption.reshape(-1, predicted_caption.size(-1))
            caption = caption.reshape(-1)
            # Calculate loss for test
            loss = criterion(predicted_caption, caption)
            total_loss_test += loss.item()

        print(f"Test Loss at epoch {epoch+1} = {total_loss_test / len(test_loader)}")


In [31]:
train_and_test(10)

Epoch 1/10 Training: 100%|██████████| 2081/2081 [45:42<00:00,  1.32s/it]


Train Loss at epoch 1 = 5.205956579801384


Epoch 1/10 Testing: 100%|██████████| 232/232 [04:49<00:00,  1.25s/it]


Test Loss at epoch 1 = 5.156835181959744


Epoch 2/10 Training: 100%|██████████| 2081/2081 [43:08<00:00,  1.24s/it]


...

Test Loss at epoch 9 = 3.625885887392636


Epoch 10/10 Training: 100%|██████████| 2081/2081 [43:18<00:00,  1.25s/it]


Train Loss at epoch 10 = 3.5681868235346563


Epoch 10/10 Testing: 100%|██████████| 232/232 [04:55<00:00,  1.27s/it]

Test Loss at epoch 10 = 3.6198415006029196


In [32]:
torch.save(encoder, "encoder_10_epochs.pth")
torch.save(decoder, "decoder_10_epochs.pth")

In [33]:
# encoder = torch.load("/kaggle/input/image-captioning/pytorch/default/1/encoder_50_epochs.pth")
# decoder = torch.load("/kaggle/input/image-captioning/pytorch/default/1/decoder_50_epochs.pth")

In [34]:
def load_img(idx):
    
    scaler = transforms.Resize([299, 299])
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    to_tensor = transforms.ToTensor()
    
    img_name = df.iloc[idx, 0]
    label = df.iloc[idx, 1]
        
    img_path = os.path.join(img_dir, img_name)
    image = Image.open(img_path)
    t_img = normalize(to_tensor(scaler(image)))
    # t_img = torch.tensor(t_img)
    # label = torch.tensor(label)
    return t_img, label

In [35]:
def inference(image, max_seq_len=50):
    result_caption = []
    image = image.to(device)
    # print(img.size())
    x = encoder(image).unsqueeze(1)
    # print(f"expected dim of input for lstm={x.size()}")
    states = None
            
    for _ in range(max_seq_len):
        hidden, states = decoder.lstm(x, states)
        output = decoder.linear(hidden)
        # print(f"output dim={output.size()}")
        predicted = output.argmax(2)
        # print(f"predicted token={predicted.item()}")
        result_caption.append(predicted.item())
        x = decoder.embed(predicted)
        # print(f"next intput size={x.size()}")
    
        if predicted.item() == 2:
            break
    
    res = [index_to_word[idx] for idx in result_caption]
    print(res)

In [36]:
img, caption = load_img(0)
# print(img.size())
img = img.unsqueeze(0)
# print(img.size())
res = inference(img)
expected = [index_to_word[idx] for idx in caption]
print(expected) 
print(res)

['<start>', '<start>', '<start>', '<start>', '<start>', 'there', 'close', 'up', 'of', 'people', 'are', 'two', 'people', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are', 'two', 'men', 'are']
['<start>', 'bicycle', 'replica', 'with', 'clock', 'as', 'the', 'front', '<end>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
None
